In [1]:
import os

os.chdir('/Users/gabriele/PycharmProjects/RecSys')
from Utils.utils import create_URM, create_ICM, combine_matrices, create_submission, write_submission
from src.Evaluation.Evaluator import EvaluatorHoldout
from src.Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
##NON-PERSONALIZED
from src.Recommenders.Base.NonPersonalizedRecommender import TopPop

#KNN
from src.Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from src.Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from src.Recommenders.KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
from src.Recommenders.KNN.ItemKNN_CFCBF_Hybrid_Recommender import ItemKNN_CFCBF_Hybrid_Recommender
from src.Recommenders.KNN.ItemKNNSimilarityHybridRecommender import ItemKNNSimilarityHybridRecommender

#MATRIX-FACTORIZATION
from src.Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from src.Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender
from src.Recommenders.MatrixFactorization.Cython.MatrixFactorization_Cython import MatrixFactorization_BPR_Cython, MatrixFactorization_FunkSVD_Cython, MatrixFactorization_AsySVD_Cython
from src.Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDRecommender
from src.Recommenders.MatrixFactorization.IALSRecommenderLinear import IALSRecommender
from src.Recommenders.MatrixFactorization.NMFRecommender import NMFRecommender

#GRAPHIC-BASED
from src.Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from src.Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender

#HYBRID
from src.Hybrids.MergingModelsByScores import MergeTwoModelsByScores, MergeThreeModelsByScores

In [2]:
##CREATION OF MATRICES AND SPLITTING IN TRAIN-TEST-VALIDATION
URM = create_URM()
ICM = create_ICM()
URM_stacked = combine_matrices(ICM=ICM, URM=URM)

In [3]:
URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM_stacked, train_percentage=0.85)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage=0.85)
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])

EvaluatorHoldout: Ignoring 13922 ( 0.4%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 13905 ( 0.6%) Users that have less than 1 test interactions


In [4]:
stacked_train = combine_matrices(ICM=ICM, URM= URM_train)

In [5]:
### RP3Beta Recommender
RP3Beta_recommender = RP3betaRecommender(URM_train=URM_train)
RP3Beta_recommender.fit(topK=181, alpha=0.5709402717259106, beta=0.3566066452521787, implicit=True, normalize_similarity=True)

In [6]:
##SLIMELASTICNET Recommender
SLIME_recommender = MultiThreadSLIM_SLIMElasticNetRecommender(URM_train=URM_stacked)
SLIME_recommender.fit(topK=420, l1_ratio=0.06061216785568925, alpha=0.00554982938879437, workers = 10)

100%|████████████████████████████████████▉| 18032/18059 [11:57<00:00, 54.65it/s]

In [7]:
similarity_slime = SLIME_recommender.W_sparse
similarity_rp3 = RP3Beta_recommender.W_sparse

In [8]:

similarity_hybrid = ItemKNNSimilarityHybridRecommender(URM_train = URM_stacked,
                                                      Similarity_1 = similarity_rp3,
                                                      Similarity_2 = similarity_slime)
similarity_hybrid.fit(topK=494, alpha=0.1597863887003789 )

In [9]:
## IALS Recommender
IALS_recommender = IALSRecommender(URM_train=URM_stacked)
IALS_recommender.fit(epochs=10, num_factors=42, alpha=0.7617528864750021, reg=8.926401306541349)

IALSRecommenderLinear: Epoch 1 of 10. Elapsed time 9.51 sec


100%|████████████████████████████████████▉| 18056/18059 [12:10<00:00, 54.65it/s]

IALSRecommenderLinear: Epoch 2 of 10. Elapsed time 16.87 sec
IALSRecommenderLinear: Epoch 3 of 10. Elapsed time 24.50 sec
IALSRecommenderLinear: Epoch 4 of 10. Elapsed time 31.79 sec
IALSRecommenderLinear: Epoch 5 of 10. Elapsed time 39.91 sec
IALSRecommenderLinear: Epoch 6 of 10. Elapsed time 47.30 sec
IALSRecommenderLinear: Epoch 7 of 10. Elapsed time 54.52 sec
IALSRecommenderLinear: Epoch 8 of 10. Elapsed time 1.04 min
IALSRecommenderLinear: Epoch 9 of 10. Elapsed time 1.23 min
IALSRecommenderLinear: Epoch 10 of 10. Elapsed time 1.41 min
IALSRecommenderLinear: Terminating at epoch 10. Elapsed time 1.41 min


In [19]:
recommenders = [similarity_hybrid, IALS_recommender]
hybrid = MergeTwoModelsByScores(URM_stacked, recommenders)
hybrid.fit(alpha=0.4555360263375717,beta=0.13654581079541908)

In [20]:
import numpy as np 
import scipy.sparse as sps

profile_length = np.ediff1d(sps.csr_matrix(URM).indptr)
profile_length, profile_length.shape

(array([168, 609, 140, ..., 260, 182, 329], dtype=int32), (13650,))

In [21]:
block_size = int(len(profile_length)*0.035)
block_size

477

In [22]:
sorted_users = np.argsort(profile_length)
sorted_users

array([1876, 9021, 5660, ..., 1141, 8526,  317])

In [23]:
cold_start_pos = 0
cold_end_pos = block_size
cold_users_group = sorted_users[cold_start_pos:cold_end_pos]

In [24]:
warm_users_group = sorted_users[cold_end_pos:]

In [25]:
len(cold_users_group)

477

In [26]:
len(warm_users_group)

13173

In [27]:
UserKNNCF_recommender = UserKNNCFRecommender(URM_train = URM_stacked)
UserKNNCF_recommender.fit(topK= 450, shrink= 5.0, similarity= 'tanimoto')

Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 13984 (100.0%), 884.96 column/sec. Elapsed time 15.80 sec


In [28]:
submission_cold =[]
for user in cold_users_group:
    submission_cold.append((user,UserKNNCF_recommender.recommend(user,10)))

In [29]:
submission_warm =[]
for user in warm_users_group:
    submission_warm.append((user,hybrid.recommend(user,10)))

100%|████████████████████████████████████▉| 18056/18059 [16:09<00:00, 18.61it/s]


In [30]:
len(submission_cold)

477

In [31]:
len(submission_warm)

13173

In [32]:
submission = submission_cold + submission_warm

In [33]:
submission.sort()

In [34]:
write_submission(submission, name="RP3hSLIME_IALS_warm_USERCF_cold")